## Faiss DB

**FAISS (Facebook AI Similarity Search)** 는 Facebook AI Research에서 개발한 고성능 벡터 유사도 검색 라이브러리입니다.

### 주요 특징
1. 빠른 벡터 검색
    - 대규모 벡터 데이터셋에서 유사한 벡터를 빠르게 찾을 수 있습니다  
    - 수백만~수십억 개의 벡터를 효율적으로 처리 가능  

2. 다양한 인덱스 타입  
    - `ndexFlatL2`: L2(유클리드) 거리 기반의 정확한 검색  
    - `IndexFlatIP`: 내적(Inner Product) 기반 검색  
    - `IndexIVFFlat`: 역색인을 사용한 근사 검색 (더 빠름)  
    - `IndexHNSW`: 계층적 그래프 기반 고속 검색  

3. 메모리 효율성  
    - 벡터 압축 기술(Product Quantization) 지원  
    - GPU 가속 지원으로 더욱 빠른 처리  

### 사용 사례  
- `추천 시스템`: 유사한 상품, 영화, 음악 추천  
- `이미지 검색`: 유사한 이미지 찾기
- `문서 검색`: 의미론적으로 유사한 문서 검색
- `RAG (Retrieval-Augmented Generation)`: LLM에서 관련 문서 검색
- `임베딩 기반 검색`: 텍스트, 이미지, 오디오 등의 벡터 표현 검색


### 장점
✅ `속도`: 매우 빠른 검색 성능  
✅ `확장성`: 대규모 데이터셋 처리 가능  
✅ `유연성`: 다양한 알고리즘과 인덱스 옵션  
✅ `C++ 기반`: 고성능 최적화, Python 바인딩 제공  

### 기본 작동 원리
- `벡터 임베딩 생성`: 텍스트/이미지를 숫자 벡터로 변환
- `인덱스 구축`: FAISS 인덱스에 벡터 추가
- `유사도 검색`: 쿼리 벡터와 가장 가까운 k개의 벡터 찾기
- `거리 계산`: L2 거리, 코사인 유사도 등으로 유사성 측정

In [2]:
# !pip install faiss-cpu

---
### 영화 추천 시스템

In [3]:
# import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

d:\PlayDataStudy\03_vectordb\39venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
movies = [
    {"id": 0, "title": "인셉션", "description": "꿈 속의 꿈을 통해 현실을 조작하는 이야기"},
    {"id": 1, "title": "매트릭스", "description": "가상 현실 속에서 인류를 구하는 해커의 모험"},
    {"id": 2, "title": "인터스텔라", "description": "인류의 생존을 위해 우주를 탐험하는 우주비행사들의 이야기"},
    {"id": 3, "title": "다크 나이트", "description": "조커와 맞서 싸우는 배트맨의 영웅적인 이야기"},
    {"id": 4, "title": "어벤져스", "description": "지구를 지키기 위해 모인 슈퍼히어로들의 활약"},
    {"id": 5, "title": "타이타닉", "description": "침몰하는 배 안에서 피어난 비극적인 사랑 이야기"},
    {"id": 6, "title": "기생충", "description": "빈부격차를 다룬 두 가족의 충격적인 만남"},
    {"id": 7, "title": "해리포터", "description": "마법 학교에서 펼쳐지는 소년 마법사의 모험"},
    {"id": 8, "title": "반지의 제왕", "description": "반지를 파괴하기 위한 장대한 판타지 여정"},
    {"id": 9, "title": "아바타", "description": "외계 행성 판도라에서 벌어지는 전쟁과 사랑"},
    {"id": 10, "title": "겨울왕국", "description": "얼음 마법을 가진 여왕과 그녀의 동생 이야기"},
    {"id": 11, "title": "토이 스토리", "description": "살아있는 장난감들의 우정과 모험"},
    {"id": 12, "title": "쇼생크 탈출", "description": "억울하게 수감된 남자의 희망과 탈출 이야기"},
    {"id": 13, "title": "포레스트 검프", "description": "순수한 남자가 겪는 미국 현대사의 여정"},
    {"id": 14, "title": "글래디에이터", "description": "노예 검투사에서 영웅이 된 로마 장군의 복수극"},
    {"id": 15, "title": "라라랜드", "description": "꿈을 향한 두 예술가의 아름답고 쓸쓸한 사랑"},
    {"id": 16, "title": "조커", "description": "광대에서 악당으로 변해가는 한 남자의 어두운 이야기"},
    {"id": 17, "title": "스파이더맨", "description": "거미에 물린 후 초능력을 얻은 학생의 영웅 이야기"},
    {"id": 18, "title": "트와일라잇", "description": "인간 소녀와 뱀파이어의 금지된 사랑"},
    {"id": 19, "title": "국제시장", "description": "한국 현대사를 관통하는 한 가장의 감동적인 인생 이야기"}
]

In [5]:
# description 임베딩
descriptions = [movie["description"] for movie in movies]

descriptions

['꿈 속의 꿈을 통해 현실을 조작하는 이야기',
 '가상 현실 속에서 인류를 구하는 해커의 모험',
 '인류의 생존을 위해 우주를 탐험하는 우주비행사들의 이야기',
 '조커와 맞서 싸우는 배트맨의 영웅적인 이야기',
 '지구를 지키기 위해 모인 슈퍼히어로들의 활약',
 '침몰하는 배 안에서 피어난 비극적인 사랑 이야기',
 '빈부격차를 다룬 두 가족의 충격적인 만남',
 '마법 학교에서 펼쳐지는 소년 마법사의 모험',
 '반지를 파괴하기 위한 장대한 판타지 여정',
 '외계 행성 판도라에서 벌어지는 전쟁과 사랑',
 '얼음 마법을 가진 여왕과 그녀의 동생 이야기',
 '살아있는 장난감들의 우정과 모험',
 '억울하게 수감된 남자의 희망과 탈출 이야기',
 '순수한 남자가 겪는 미국 현대사의 여정',
 '노예 검투사에서 영웅이 된 로마 장군의 복수극',
 '꿈을 향한 두 예술가의 아름답고 쓸쓸한 사랑',
 '광대에서 악당으로 변해가는 한 남자의 어두운 이야기',
 '거미에 물린 후 초능력을 얻은 학생의 영웅 이야기',
 '인간 소녀와 뱀파이어의 금지된 사랑',
 '한국 현대사를 관통하는 한 가장의 감동적인 인생 이야기']

In [6]:
# description 임베딩 : np array 형태로 변환
embeddings = np.array([model.encode(description) for description in descriptions], dtype='float32')

embeddings

array([[ 1.26796672e-02,  8.19030479e-02,  3.63245010e-02, ...,
         3.17518525e-02, -7.26569146e-02, -2.80585401e-02],
       [ 1.90406851e-02,  3.53296511e-02,  4.15985845e-02, ...,
         5.10477228e-03, -4.10996266e-02,  5.64820832e-04],
       [ 3.03830579e-02,  7.42852222e-03, -5.10917185e-03, ...,
        -3.38630602e-02,  5.18163579e-05,  1.34261763e-02],
       ...,
       [ 3.64420228e-02,  2.85018384e-02,  3.27093005e-02, ...,
        -2.02104393e-02, -4.21426669e-02,  2.18916014e-02],
       [ 3.78947370e-02,  3.70906107e-02,  2.98954975e-02, ...,
         1.06996512e-02, -5.47157638e-02, -1.38526252e-02],
       [ 3.36428024e-02,  6.96328431e-02,  3.07361167e-02, ...,
         2.17385385e-02, -1.20583944e-01,  2.79705375e-02]],
      shape=(20, 384), dtype=float32)

In [ ]:
import faiss

dim = embeddings.shape[1]  # 임베딩 벡터의 차원

# faiss.IndexFlatL2 : L2 거리 기반의 인덱스 생성 ( = 데이터 구조 )
# - 벡터 검색을 효율적으로 수행하기 위한 데이터 구조
# - 유클리드 거리(L2 거리)를 사용하여 벡터 간의 유사성을 측정
faiss_index = faiss.IndexFlatL2(dim)  
faiss_index.add(embeddings)  # 임베딩 벡터들을 인덱스에 추가

In [8]:
# 질의 텍스트 임베딩
query = "우주를 여행하는 이야기"
query_embedding = np.array([model.encode(query)], dtype='float32')

In [11]:
# index 검색
# - distances: 질의 벡터와 검색된 벡터들 간의 거리
# - indices: 검색된 벡터들의 인덱스

k = 3
distances, indices = faiss_index.search(query_embedding, k=k)  # k는 상위 몇 개의 유사한 항목을 찾을지 지정

distances, indices

(array([[0.320683 , 0.3257113, 0.3899225]], dtype=float32),
 array([[ 0, 16, 13]]))

In [12]:
for i in range(k):
    movie_index = indices[0][i]
    print(f"{i+1}번째 추천: {movies[movie_index]['title']} - {movies[movie_index]['description']}")

1번째 추천: 인셉션 - 꿈 속의 꿈을 통해 현실을 조작하는 이야기
2번째 추천: 조커 - 광대에서 악당으로 변해가는 한 남자의 어두운 이야기
3번째 추천: 포레스트 검프 - 순수한 남자가 겪는 미국 현대사의 여정


---
### 사용자 맞춤 뉴스 추천

In [ ]:
# !pip install BeautifulSoup

In [3]:
import requests
from bs4 import BeautifulSoup


# 1. url requests 요청
keyword = input("뉴스 검색 키워드 입력 : ")
domain = "https://search.naver.com/search.naver?ssc=tab.news.all&where=news&sm=tab_jum&query="
url = f"{domain}{keyword}"

# 2. response 받기
response = requests.get(url) 
html = response.text


# 3. BeautifulSoup 파싱
soup = BeautifulSoup(response.text, "html.parser")

# 4. 데이터 추출
news_content = soup.select(".fds-news-item-list-tab > div")


news_list = []

for idx, news in enumerate(news_content):
    title = news.select_one(".sds-comps-text-type-headline1").text 
    news_list.append({"title": title})



In [5]:
# 임베딩 모델 로드 및 임베딩 처리
import numpy as np
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2') 
embeddings = np.array([model.encode(news['title']) for news in news_list], dtype='float32')

embeddings

d:\PlayDataStudy\03_vectordb\39venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


array([[-2.93730740e-02, -3.92229529e-03,  1.29503589e-02, ...,
        -2.19790768e-02, -3.76406079e-03,  3.25353257e-02],
       [-3.24576162e-02,  8.09813440e-02,  4.89119068e-02, ...,
        -1.00710765e-02, -1.46819605e-03,  1.07982986e-01],
       [ 2.50749420e-02,  7.04471245e-02, -8.53759609e-03, ...,
        -1.56355221e-02, -7.20788687e-02,  2.44992971e-02],
       ...,
       [-1.35690719e-03,  5.65075576e-02, -5.42205060e-04, ...,
        -2.13777255e-02, -6.18335567e-02,  2.16417294e-02],
       [ 5.09948172e-02, -7.64978211e-03,  3.03397067e-02, ...,
        -3.86506617e-02, -1.31497653e-02,  3.77001651e-02],
       [-1.12886680e-02,  2.14189813e-02, -4.77304459e-02, ...,
         8.18604603e-03, -6.54508965e-03,  1.33555432e-05]],
      shape=(10, 384), dtype=float32)

In [7]:
# FAISS index 생성 및 데이터 추가
import faiss

dim = embeddings.shape[1] 
faiss_index = faiss.IndexFlatL2(dim)  
faiss_index.add(embeddings)

In [8]:
# 질의 텍스트 임베딩
query = "최근 화재 사건"
query_embedding = np.array([model.encode(query)], dtype='float32')

In [12]:
# index 검색
k = 3
distances, indices = faiss_index.search(query_embedding, k=k)  # k는 상위 몇 개의 유사한 항목을 찾을지 지정

for i in range(k):
    news_index = indices[0][i] 
    print(f"{i+1}번째 추천 뉴스: {news_list[news_index]['title']} (유사도 거리: {distances[0][i]:.2f}) ")

1번째 추천 뉴스: 대구 남구 주택서 화재…60대 남성 사망 (유사도 거리: 0.69) 
2번째 추천 뉴스: 양주 대형마트 화재…소방당국 진화 중 (유사도 거리: 0.77) 
3번째 추천 뉴스: 포항 개 사육장 화재…개 6마리 불에 타 죽어 (유사도 거리: 0.91) 
